# Copyright Netherlands eScience Center <br>
**Function     : Paper Fig - address uncertainty with BayesianLSTM and the related metrics** <br>
**Author       : Yang Liu** <br>
**First Built  : 2020.07.08** <br>
**Last Update  : 2020.07.08** <br>
**Library      : Pytorth, Numpy, NetCDF4, os, iris, cartopy, dlacs, matplotlib**<br>
Description     : This notebook serves to make figures for the paper.<br>

The Lorens-84 model is described by Edward Lorens in his 1984 paper:<br>
Lorenz, E. N. (1984). Irregularity: A fundamental property of the atmosphere. Tellus A, 36(2), 98-110.<br>

Return Values   : Time series and figures <br>

In [1]:
%matplotlib inline

import sys
import warnings
import numbers

# for data loading
import os
from netCDF4 import Dataset
# for pre-processing and machine learning
import numpy as np
import csv
#import sklearn
#import scipy
import torch
import torch.nn.functional

#sys.path.append(os.path.join('C:','Users','nosta','ML4Climate','Scripts','DLACs'))
sys.path.append("C:\\Users\\nosta\\ML4Climate\\Scripts\\DLACs")
#sys.path.append("../")
import dlacs
import dlacs.BayesConvLSTM
import dlacs.preprocess
import dlacs.function
import dlacs.saveNetCDF
import dlacs.metric

# for visualization
import dlacs.visual
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from mpl_toolkits.mplot3d import Axes3D
import iris # also helps with regriding
import cartopy
import cartopy.crs as ccrs

# ignore all the DeprecationWarnings by pytorch
if not sys.warnoptions:
    warnings.simplefilter("ignore")
###############################################################################79

The testing device is Dell Inspirion 5680 with Intel Core i7-8700 x64 CPU and Nvidia GTX 1060 6GB GPU.<br>
Here is a benchmark about cpu v.s. gtx 1060 <br>
https://www.analyticsindiamag.com/deep-learning-tensorflow-benchmark-intel-i5-4210u-vs-geforce-nvidia-1060-6gb/

In [2]:
# constants
constant = {'g' : 9.80616,      # gravititional acceleration [m / s2]
            'R' : 6371009,      # radius of the earth [m]
            'cp': 1004.64,      # heat capacity of air [J/(Kg*K)]
            'Lv': 2500000,      # Latent heat of vaporization [J/Kg]
            'R_dry' : 286.9,    # gas constant of dry air [J/(kg*K)]
            'R_vap' : 461.5,    # gas constant for water vapour [J/(kg*K)]
            'rho' : 1026,       # sea water density [kg/m3]
            }

In [3]:
################################################################################# 
#########                           datapath                             ########
#################################################################################
ಠ_ಠ = "I'm a valid variable name!"
print(ಠ_ಠ)
# model type
model_type = 'lorenz84ex'
# datapath for output
output_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps\\Lorenz84\\bayes\\fullseries_ex\\paperFig'
# data path of experiment
pred_lorenz_bayesconvlstm_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps\\Lorenz84\\bayes\\fullseries_ex\\penalty100'

I'm a valid variable name!


In [4]:
if __name__=="__main__":
    #################################################################################
    ###########                configure Lorenz 84 model                  ###########
    #################################################################################
    # Lorenz paramters and initial conditions
    x_init = 1.0 # strength of the symmetric globally encircling westerly current
    y_init = 1.0 # strength of the cosine phases of a chain of superposedwaves (large scale eddies)
    z_init = 1.0 # strength of the sine phases of a chain of superposedwaves (large scale eddies)
    F = 8.0 # temperature difference between pole and equator
    G = 1.0 # thermal forcing term
    epsilon = 0.4 # intensity of periodic forcing
    a = 0.25 # mechanical damping factor for westerly wind x
    b = 4.0 # advection strength of the waves by the westerly current
    
    # assuming the damping time for the waves is 5 days (Lorens 1984)
    dt = 0.0333 # 1/30 unit of time unit (5 days)
    num_steps = 1500
    # cut-off point of initialization period
    cut_off = 0
    # target testing period
    test_len = 200

#### Ensemble<br>
##### Testing benchmark <br>
- Lorenz 84 model initial set-up <br>
original model<br>
**x=1.0, y=1.0, z=1.0, a=0.25, b=4.0, F=8.0, G=1.0** <br>
with periodic forcing<br>
**x=1.0, y=1.0, z=1.0, a=0.25, b=4.0, F=8.0, G=1.0, epsilon=0.4** <br>
with AA effect<br>
**x=1.0, y=1.0, z=1.0, a=0.25, b=4.0, F=8.0, G=1.0, epsilon=0.4, alpha = 0.025, beta = 0.00625** <br>

##### Testing 
- Testing initial conditions x, y, z - uncertainty of model outcome <br>
**starting point x=1.0, y=1.0, z=1.0** <br>
Ensemble variant x 50 members <br>
x = [1.0020 : 0.9980] **~0.2%**<br>
x = [1.0010 : 0.9990] **~0.1%**<br>
x = [1.00010 : 0.99990] **~0.01%**<br>
x_ex = [1.0010 : 0.9990] **~0.1%**<br>
x_ex = [1.00010 : 0.99990] **~0.01%**<br>
x_ex = [1.000010 : 0.999990] **~0.001%**<br>
========================================================================================================<br>
- Testing model parameters a & b - model uncertainty <br>
**starting point a=0.25, b=4.0** <br>
Ensemble variant a <br>
a = [0.25010 : 0.24990] **~0.04%**<br>
a_ex = [0.250010 : 0.249990] **~0.004%**<br>
a_ex = [0.2500010 : 0.2499990] **~0.0004%**<br>
========================================================================================================<br>

In [5]:
    #################################################################################
    ###########            Lorens 84 model + periodic forcing             ###########
    #################################################################################
    def lorenz84_ex(x, y, z, t, a = 0.25, b = 4.0, F = 8.0, G = 1.0, epsilon = 0.4):
        """
        Solver of Lorens-84 model with periodic external forcing.
        
        param x, y, z: location in a 3D space
        param a, b, F, G: constants and forcing
        
        The model is designed with a reference to the paper:
        Broer, H., Simó, C., & Vitolo, R. (2002). Bifurcations and strange
        attractors in the Lorenz-84 climate model with seasonal forcing. Nonlinearity, 15(4), 1205.
        
        Song, Y., Yu, Y., & Wang, H. (2011, October). The stability and chaos analysis of the
        Lorenz-84 atmosphere model with seasonal forcing. In 2011 Fourth International Workshop
        on Chaos-Fractals Theories and Applications (pp. 37-41). IEEE.
        """
        # each time step is ~ 5days, therefore the returning period are 365 / 5 = 73 times in a year
        T = 73
        omega = 2 * np.pi / T
        dx = - y**2 - z**2 - a * x + a * F * (1 + epsilon * np.cos(omega * t))
        dy = x * y - b * x * z - y + G * (1 + epsilon * np.sin(omega * t))
        dz = b * x * y + x * z - z
        
        return dx, dy, dz

In [6]:
    #################################################################################
    ###########        Launch Lorenz 84 model with periodic forcing       ###########
    #################################################################################
    # Need one more for the initial values
    x = np.empty(num_steps)
    y = np.empty(num_steps)
    z = np.empty(num_steps)
    t = 0.0
    
    # save initial values
    x[0] = x_init
    y[0] = y_init
    z[0] = z_init
    
    # Step through "time", calculating the partial derivatives at the current point
    # and using them to estimate the next point
    for i in range(num_steps-1):
        dx, dy, dz = lorenz84_ex(x[i], y[i], z[i], t, a, b ,F, G, epsilon)
        x[i + 1] = x[i] + (dx * dt)
        y[i + 1] = y[i] + (dy * dt)
        z[i + 1] = z[i] + (dz * dt)
        t += dt

In [7]:
    print ('*******************  pre-processing  *********************')
    # time series cut-off
    if cut_off:
        x = x[cut_off:]
        y = y[cut_off:]
        z = z[cut_off:]
    print ('===================  normalize data  =====================')
    x_norm = dlacs.preprocess.operator.normalize(x)
    y_norm = dlacs.preprocess.operator.normalize(y)
    z_norm = dlacs.preprocess.operator.normalize(z)
    print('================  save the normalizing factor  =================')
    # maximum and minimum
    x_max = np.amax(x)
    x_min = np.amin(x)
    y_max = np.amax(y)
    y_min = np.amin(y)
    z_max = np.amax(z)
    z_min = np.amin(z)
    # std
    x_std = np.std(x)
    y_std = np.std(y)
    z_std = np.std(z)

*******************  pre-processing  *********************
===================  normalize data  =====================
================  save the normalizing factor  =================


In [8]:
    #################################################################################
    #########        perturb initial condition x and model parameter a       ########
    #################################################################################
    x_pool_per = ["0.001%", "0.01%", "0.1%", "1.0%"]
    a_pool_per = ["0.001%", "0.01%", "0.1%", "1.0%"]
    #################################################################################
    ###########                 perturb model parameters                  ###########
    #################################################################################
    # percentage of perturbation
    x_pool = np.array([0.00001, 0.0001, 0.001, 0.01], dtype=float)
    a_pool = np.array([0.00001, 0.0001, 0.001, 0.01], dtype=float)
    
    # perturbation of parameters
    x_init_pert = x_pool * x_init + x_init
    a_init_pert = - a_pool * a + a 

# Postprocess forecast data <br>
** Postprocess and visualization of ensemble forecast data ** <br>

In [9]:
    #######################################################################
    #######    Load forecast of Lorenz 84 model by BayesConvLSTM    #######
    #######################################################################
    print ('*******************  load ensemble data  *********************')
    # predict x steps ahead
    step_lead = 16 # unit week
    # number of ensembles
    ens = 20
    predictand = np.zeros((ens, 3, test_len, step_lead), dtype=float)
    for i in range(ens):
        dataset_pred = Dataset(os.path.join(pred_lorenz_bayesconvlstm_path,
                                            'pred_lorenz84_ens_{}.nc'.format(i)))
        predictand[i,:,:,:] = dataset_pred.variables['series'][:]
    print(predictand.shape)
    predictand_median = np.median(predictand, 0)

*******************  load ensemble data  *********************
(20, 3, 200, 16)


In [10]:
    ###################################################################################################
    ###########        Launch Lorenz 84 model with periodic forcing with longer series      ###########
    ###################################################################################################
    num_steps_long = num_steps + step_lead
    # Need one more for the initial values
    xx = np.empty(num_steps_long)
    yy = np.empty(num_steps_long)
    zz = np.empty(num_steps_long)
    tt = 0.0
    
    # save initial values
    xx[0] = x_init
    yy[0] = y_init
    zz[0] = z_init
    
    # Step through "time", calculating the partial derivatives at the current point
    # and using them to estimate the next point
    for i in range(num_steps_long-1):
        dx, dy, dz = lorenz84_ex(xx[i], yy[i], zz[i], tt, a, b ,F, G, epsilon)
        xx[i + 1] = xx[i] + (dx * dt)
        yy[i + 1] = yy[i] + (dy * dt)
        zz[i + 1] = zz[i] + (dz * dt)
        tt += dt

In [11]:
    # Euclidean distance
    def Euclidean(obs, pred):
        """
        Measure the Euclidean distance.
        ----------------------
        param obs: observation time series with the shape [dim, series]
        param pred: reference time series with the shape [dim, series]        
        """
        euclidean = np.mean(np.sqrt(np.sum((obs-pred)**2,0)))
        
        return euclidean

    # RMSE
    def RMSE(obs, pred):
        """
        Root Mean Square Error.
        """
        rmse = np.sqrt(np.mean((obs - pred)**2))
        
        return rmse

In [12]:
    ###########################################################################################
    ##########       generate model output with perturbed data and compute RMSE      ##########
    ###########################################################################################    
    rmse_pert_x = np.zeros((len(x_init_pert), 3), dtype=float)
    rmse_pert_a = np.zeros((len(a_init_pert), 3), dtype=float)
    
    euclidean_pert_x = np.zeros((len(x_init_pert)), dtype=float)
    euclidean_pert_a = np.zeros((len(a_init_pert)), dtype=float)
    
    # for euclidean distance
    obs = np.zeros((3, num_steps), dtype=float)
    obs[0,:] = xx[:-step_lead]
    obs[1,:] = yy[:-step_lead]
    obs[2,:] = zz[:-step_lead]
    
    for n in range(len(x_init_pert)):
        print("Perturb the paramter x by {}".format(x_pool_per[n]))
        #################################################################################
        ###########        Launch Lorenz 84 model with periodic forcing       ###########
        #################################################################################
        # Need one more for the initial values
        x = np.empty(num_steps)
        y = np.empty(num_steps)
        z = np.empty(num_steps)
        t = 0.0
        
        x[0] = x_init_pert[n]
        y[0] = y_init
        z[0] = z_init
        
        for i in range(num_steps-1):
            dx, dy, dz = lorenz84_ex(x[i], y[i], z[i], t, a, b ,F, G, epsilon)
            x[i + 1] = x[i] + (dx * dt)
            y[i + 1] = y[i] + (dy * dt)
            z[i + 1] = z[i] + (dz * dt)
            t += dt
        
        rmse_pert_x[n,0] = RMSE(xx[-(test_len + step_lead):-step_lead], x[-test_len:])
        rmse_pert_x[n,1] = RMSE(yy[-(test_len + step_lead):-step_lead], y[-test_len:])
        rmse_pert_x[n,2] = RMSE(zz[-(test_len + step_lead):-step_lead], z[-test_len:])
        
        obs_perturb = np.zeros((3, num_steps), dtype=float)
        obs_perturb[0,:] = x[:]
        obs_perturb[1,:] = y[:]
        obs_perturb[2,:] = z[:]        
        
        euclidean_pert_x[n] = Euclidean(obs[:,-(test_len + step_lead):-step_lead], obs_perturb[:,-test_len:])
        
    print(rmse_pert_x)
    print(euclidean_pert_x)
    
    for n in range(len(a_init_pert)):
        print("Perturb the paramter a by {}".format(a_pool_per[n]))
        #################################################################################
        ###########        Launch Lorenz 84 model with periodic forcing       ###########
        #################################################################################
        # Need one more for the initial values
        x = np.empty(num_steps)
        y = np.empty(num_steps)
        z = np.empty(num_steps)
        t = 0.0
        
        aa = a_init_pert[n]
        
        x[0] = x_init
        y[0] = y_init
        z[0] = z_init
        
        for i in range(num_steps-1):
            dx, dy, dz = lorenz84_ex(x[i], y[i], z[i], t, aa, b ,F, G, epsilon)
            x[i + 1] = x[i] + (dx * dt)
            y[i + 1] = y[i] + (dy * dt)
            z[i + 1] = z[i] + (dz * dt)
            t += dt
        
        rmse_pert_a[n,0] = RMSE(xx[-(test_len + step_lead):-step_lead], x[-test_len:])
        rmse_pert_a[n,1] = RMSE(yy[-(test_len + step_lead):-step_lead], y[-test_len:])
        rmse_pert_a[n,2] = RMSE(zz[-(test_len + step_lead):-step_lead], z[-test_len:])
        
        obs_perturb = np.zeros((3, num_steps), dtype=float)
        obs_perturb[0,:] = x[:]
        obs_perturb[1,:] = y[:]
        obs_perturb[2,:] = z[:]        
        
        euclidean_pert_a[n] = Euclidean(obs[:,-(test_len + step_lead):-step_lead], obs_perturb[:,-test_len:])

    print(rmse_pert_a)
    print(euclidean_pert_a)

Perturb the paramter x by 0.001%
Perturb the paramter x by 0.01%
Perturb the paramter x by 0.1%
Perturb the paramter x by 1.0%
[[0.13043759 0.51952454 0.5150073 ]
 [0.30267206 1.02412727 1.0275891 ]
 [0.43047495 1.13196735 1.17672187]
 [0.38873922 1.08647351 1.04677691]]
[0.7316916  0.27085398 0.99589553 1.41996103]
Perturb the paramter a by 0.001%
Perturb the paramter a by 0.01%
Perturb the paramter a by 0.1%
Perturb the paramter a by 1.0%
[[0.06548156 0.26737304 0.26480829]
 [0.30924268 1.03608308 1.04067395]
 [0.31879982 1.05144448 1.05763518]
 [0.44599381 1.09287441 1.01849402]]
[0.977951   0.30703509 0.35715895 1.42490342]


In [13]:
    ##############################################################################################################
    ###########       Compute RMSE with given ensemble predictands and deterministic observation       ###########
    ##############################################################################################################
    # lead time dependent RMSE of Bayesian LSTM forecast
    rmse_pred = np.zeros((ens, step_lead, 3), dtype=float)
    for i in range(step_lead):
        for n in range(ens):
            rmse_x_pred = RMSE(xx[-(test_len + step_lead)+i:-step_lead+i], predictand[n,0,:,i]* (x_max - x_min) + x_min)
            rmse_y_pred = RMSE(yy[-(test_len + step_lead)+i:-step_lead+i], predictand[n,1,:,i]* (y_max - y_min) + y_min)
            rmse_z_pred = RMSE(zz[-(test_len + step_lead)+i:-step_lead+i], predictand[n,2,:,i]* (z_max - z_min) + z_min)
            
            rmse_pred[n,i,0] = rmse_x_pred
            rmse_pred[n,i,1] = rmse_y_pred
            rmse_pred[n,i,2] = rmse_z_pred
            
    rmse_pred_mean = np.mean(rmse_pred, 0)
    
    # lead time dependent RMSE of persistence
    rmse_persist = np.zeros((step_lead,3), dtype=float)
    for i in range(step_lead):
        rmse_persist[i,0] = RMSE(xx[-(test_len + step_lead)+i:-step_lead+i], xx[-(test_len + step_lead)-1:-step_lead-1])
        rmse_persist[i,1] = RMSE(yy[-(test_len + step_lead)+i:-step_lead+i], yy[-(test_len + step_lead)-1:-step_lead-1])
        rmse_persist[i,2] = RMSE(zz[-(test_len + step_lead)+i:-step_lead+i], zz[-(test_len + step_lead)-1:-step_lead-1])

In [14]:
    ##############################################################################################################
    ###########       Compute Euclidean distance with given ensemble predictands and observation       ###########
    ##############################################################################################################
    # lead time dependent Euclidean distance of Bayesian forecast
    euclidean_pred_ens = np.zeros((ens, step_lead), dtype=float)
    
    predictand_pred_denorm = np.zeros(predictand.shape, dtype=float)
    predictand_pred_denorm[:,0,:,:] = predictand[:,0,:,:] * (x_max - x_min) + x_min
    predictand_pred_denorm[:,1,:,:] = predictand[:,1,:,:] * (y_max - y_min) + y_min
    predictand_pred_denorm[:,2,:,:] = predictand[:,2,:,:] * (z_max - z_min) + z_min
    
    for i in range(step_lead):
        for n in range(ens):
            euclidean_pred_n = Euclidean(obs[:,-(test_len + step_lead)+i:-step_lead+i], predictand_pred_denorm[n,:,:,i])
            euclidean_pred_ens[n,i] = euclidean_pred_n
            
    euclidean_pred = np.mean(euclidean_pred_ens, 0)
    
    # lead time dependent Euclidean distance of persistence
    euclidean_persist = np.zeros(step_lead, dtype=float)
    for i in range(step_lead):
        euclidean_persist[i] = Euclidean(obs[:,-(test_len + step_lead)+i:-step_lead+i], obs[:,-(test_len + step_lead)-1:-step_lead-1])

In [17]:
    # Write CSV file
    with open(os.path.join(output_path, "RMSE_perturb.csv"), "wt+") as fp:
        writer = csv.writer(fp, delimiter=",")
        for n in range(len(x_init_pert)):
            writer.writerow(["x", "y", "z", "perturb x {}".format(x_init_pert[n])])  # write header
            writer.writerow(rmse_pert_x[n,:].T)
        for n in range(len(a_init_pert)):
            writer.writerow(["x", "y", "z", "perturb a {}".format(a_init_pert[n])])  # write header
            writer.writerow(rmse_pert_a[n,:].T)            
        writer.writerow(["x", "y", "z", "Forecast"])  # write header
        writer.writerow(np.arange(step_lead))  # write header
        writer.writerow(rmse_pred_mean.T)
        writer.writerow(["x", "y", "z", "Persistence"])  # write header
        writer.writerow(np.arange(step_lead))  # write header
        writer.writerow(rmse_persist.T)   
    
    with open(os.path.join(output_path, "EuclideanD_perturb.csv"), "wt+") as fp:
        writer = csv.writer(fp, delimiter=",")
        for n in range(len(x_init_pert)):
            writer.writerow(["Euclidean", "perturb x {}".format(x_init_pert[n])])  # write header
            writer.writerow([euclidean_pert_x[n]])
        for n in range(len(a_init_pert)):
            writer.writerow(["Euclidean", "perturb a {}".format(a_init_pert[n])])  # write header
            writer.writerow([euclidean_pert_a[n]])           
        writer.writerow(["Euclidean", "Forecast"])  # write header
        writer.writerow(np.arange(step_lead))  # write header
        writer.writerow(euclidean_pred)
        writer.writerow(["Euclidean", "Persistence"])  # write header
        writer.writerow(np.arange(step_lead))  # write header
        writer.writerow(euclidean_persist)